In [19]:
import pandas as pd
import numpy as np


# Import Results

In [20]:
results_df = pd.read_csv('/Users/kristianertresvag/Documents/Handelshøyskolen BI/Master/4. semester/Master Thesis/Master Thesis/Analysis/results.csv')

results_df

,Unnamed: 0,Series,Date,TimeGPT,Chronos,Moirai,ForecastPeriod,ARARMA,Auto-ANN,AutoBox1,...,PP-Autocast,RBF,ROBUST-Trend,SMARTFCS,THETA,THETAsm,WINTER,Actual,Category,NF
0,0,1,1989-01-01,5121.754395,5471.703660,5075.0400,1,5553.26,5453.28,5442.92,...,5475.80,5550.22,5204.32,5259.00,5414.60,5163.08,5406.43,5379.75,MICRO,6
1,1,1,1990-01-01,4974.531738,5847.765768,5983.5040,2,6246.45,6003.49,5969.23,...,6014.62,6152.23,5471.66,5675.00,5934.47,5390.88,5875.96,6158.68,MICRO,6
2,2,1,1991-01-01,4750.660156,6195.623217,5960.6943,3,7026.18,6587.93,6495.55,...,6553.43,6648.95,5738.99,6091.00,6331.96,5155.17,6345.48,6876.58,MICRO,6
3,3,1,1992-01-01,4366.261719,6195.623217,6046.6260,4,7903.23,7206.76,7021.87,...,7092.24,6965.22,6006.32,6507.00,6822.35,5147.46,6815.00,7851.91,MICRO,6
4,4,1,1993-01-01,3914.425781,5866.568873,7204.8813,5,8889.77,7860.01,7548.18,...,7631.05,7038.06,6273.66,6923.00,7140.76,5152.92,7284.53,8407.84,MICRO,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35617,35617,2829,1988-07-01,1437.689209,1259.141011,1307.1370,14,1335.28,1139.87,1254.13,...,1384.45,1371.58,1223.93,1170.98,1226.58,1452.61,1193.75,1282.50,OTHER,18
35618,35618,2829,1988-08-01,1442.109741,1243.785633,1468.1404,15,1316.26,1116.26,1234.02,...,1380.21,1355.06,1203.67,1147.83,1207.17,1451.93,1171.28,1261.30,OTHER,18
35619,35619,2829,1988-09-01,1439.912842,1243.785633,1361.4963,16,1307.62,1091.50,1213.86,...,1376.36,1345.82,1183.41,1124.68,1187.85,1448.36,1148.82,1263.40,OTHER,18
35620,35620,2829,1988-10-01,1437.093018,1243.785633,1473.0730,17,1293.64,1066.18,1193.64,...,1372.85,1336.08,1163.14,1101.53,1168.63,1458.19,1126.35,1257.10,OTHER,18


# Calculate MdAPE

In [21]:
# Function to calculate MdAPE
def calculate_mdape(forecast, actual):
    ape_elements = np.abs((forecast - actual) / actual) * 100
    mdape = np.median(ape_elements)
    return round(mdape, 2)

forecast_cols = [col for col in results_df.columns if col not in ['Unnamed: 0', 'Series', 'Date', 'ForecastPeriod', 'Category', 'NF', 'Actual']]

mdape_values = pd.DataFrame(columns=['Method', 'ForecastPeriod', 'MdAPE'])

results = []
for method in forecast_cols:
    for period in sorted(results_df['ForecastPeriod'].unique()):
        actual_values = results_df[results_df['ForecastPeriod'] == period]['Actual'].values
        forecast_values = results_df[results_df['ForecastPeriod'] == period][method].values
        mdape = calculate_mdape(forecast_values, actual_values)
        results.append({'Method': method, 'ForecastPeriod': period, 'MdAPE': mdape})

mdape_values = pd.DataFrame(results)
mdape_pivot_table = mdape_values.pivot(index='Method', columns='ForecastPeriod', values='MdAPE')

for col_range in [(1, 4), (1, 6), (1, 8), (1, 12), (1, 15), (1, 18)]:
    cols = [col for col in range(col_range[0], col_range[1] + 1)]
    col_name = f"Avg {col_range[0]}-{col_range[1]}"
    mdape_pivot_table[col_name] = round(mdape_pivot_table[cols].mean(axis=1), 2)

unique_series_count = {method: results_df.groupby(method)['Series'].nunique() for method in forecast_cols}

mdape_pivot_table['#obs'] = np.max(results_df['Series'])

custom_order = [
    'NAIVE2',
    'HOLT',
    'DAMPEN',
    'WINTER',
    'COMB S-H-D',
    'B-J auto',
    'AutoBox1',
    'AutoBox2',
    'AutoBox3',
    'ROBUST-Trend',
    'ARARMA',
    'Auto-ANN',
    'Flors-Pearc1',
    'Flors-Pearc2',
    'PP-Autocast',
    'ForecastPro',
    'SMARTFCS',
    'THETAsm',
    'THETA',
    'RBF',
    'ForcX',
    'Chronos',
    'TimeGPT',
    'Moirai'
]

mdape_pivot_table = mdape_pivot_table.reindex(custom_order)

mdape_pivot_table.to_csv('mdape_table.csv')

mdape_pivot_table


ForecastPeriod,1,2,3,4,5,6,7,8,9,10,...,16,17,18,Avg 1-4,Avg 1-6,Avg 1-8,Avg 1-12,Avg 1-15,Avg 1-18,#obs
Method,,,,,,,,,,,,,,,,,,,,,
NAIVE2,3.66,4.75,6.05,7.06,7.57,8.47,7.67,7.70,7.82,8.22,...,10.92,10.82,11.62,5.38,6.26,6.62,7.10,7.62,8.20,2829
HOLT,3.35,4.33,5.39,6.59,7.19,7.42,6.60,6.55,6.43,7.46,...,9.13,9.29,10.32,4.92,5.71,5.93,6.29,6.66,7.15,2829
DAMPEN,3.28,4.04,5.10,6.51,6.79,7.42,6.81,6.88,6.73,6.80,...,10.40,9.75,10.51,4.73,5.52,5.85,6.21,6.72,7.30,2829
WINTER,3.42,4.37,5.50,6.51,7.25,7.49,6.66,6.84,6.50,7.23,...,9.26,9.36,10.14,4.95,5.76,6.01,6.32,6.75,7.22,2829
COMB S-H-D,3.35,4.18,5.15,6.47,6.80,7.19,6.58,6.70,6.43,6.84,...,9.39,9.13,9.83,4.79,5.52,5.80,6.10,6.58,7.06,2829
B-J auto,3.29,4.23,5.37,6.60,6.78,7.26,6.56,6.48,6.86,6.94,...,10.44,9.65,10.65,4.87,5.59,5.82,6.18,6.68,7.27,2829
AutoBox1,3.72,4.66,5.75,6.61,7.22,7.93,6.64,7.03,6.45,6.76,...,9.73,9.75,10.55,5.18,5.98,6.20,6.42,6.86,7.38,2829
AutoBox2,3.39,4.41,5.22,6.35,6.43,7.23,6.78,6.36,6.77,6.71,...,9.99,9.40,10.26,4.84,5.50,5.77,6.13,6.66,7.20,2829
AutoBox3,3.62,4.74,5.70,6.80,7.17,7.74,6.87,7.27,6.90,7.03,...,10.05,10.31,10.45,5.22,5.96,6.24,6.49,6.95,7.50,2829


# Calculate MdRAE

In [22]:
# Function to calculate MdRAE
def calculate_mdrae(forecast, actual):
    numerator = np.abs(forecast - actual)
    denominator = np.abs(actual - np.median(actual))
    denominator = np.where(denominator == 0, 0.001, denominator)
    drae_elements = numerator / denominator
    mdrae = np.median(drae_elements)
    return round(mdrae, 2)

forecast_cols = [col for col in results_df.columns if col not in ['Unnamed: 0', 'Series', 'Date', 'ForecastPeriod', 'Category', 'NF', 'Actual']]

mdrae_values = pd.DataFrame(columns=['Method', 'ForecastPeriod', 'MdRAE'])

results = []
for method in forecast_cols:
    for period in sorted(results_df['ForecastPeriod'].unique()):
        actual_values = results_df[results_df['ForecastPeriod'] == period]['Actual'].values
        forecast_values = results_df[results_df['ForecastPeriod'] == period][method].values
        mdrae = calculate_mdrae(forecast_values, actual_values)
        results.append({'Method': method, 'ForecastPeriod': period, 'MdRAE': mdrae})

mdrae_values = pd.DataFrame(results)
mdrae_pivot_table = mdrae_values.pivot(index='Method', columns='ForecastPeriod', values='MdRAE')

for col_range in [(1, 4), (1, 6), (1, 8), (1, 12), (1, 15), (1, 18)]:
    cols = [col for col in range(col_range[0], col_range[1] + 1)]
    col_name = f"Avg {col_range[0]}-{col_range[1]}"
    mdrae_pivot_table[col_name] = round(mdrae_pivot_table[cols].mean(axis=1), 2)

unique_series_count = {method: results_df.groupby(method)['Series'].nunique() for method in forecast_cols}

mdrae_pivot_table['#obs'] = np.max(results_df['Series'])

custom_order = [
    'NAIVE2',
    'HOLT',
    'DAMPEN',
    'WINTER',
    'COMB S-H-D',
    'B-J auto',
    'AutoBox1',
    'AutoBox2',
    'AutoBox3',
    'ROBUST-Trend',
    'ARARMA',
    'Auto-ANN',
    'Flors-Pearc1',
    'Flors-Pearc2',
    'PP-Autocast',
    'ForecastPro',
    'SMARTFCS',
    'THETAsm',
    'THETA',
    'RBF',
    'ForcX',
    'Chronos',
    'TimeGPT',
    'Moirai'
]

mdrae_pivot_table = mdrae_pivot_table.reindex(custom_order)

mdrae_pivot_table.to_csv('mdrae_table.csv')

mdrae_pivot_table

ForecastPeriod,1,2,3,4,5,6,7,8,9,10,...,16,17,18,Avg 1-4,Avg 1-6,Avg 1-8,Avg 1-12,Avg 1-15,Avg 1-18,#obs
Method,,,,,,,,,,,,,,,,,,,,,
NAIVE2,0.18,0.20,0.25,0.31,0.32,0.35,0.32,0.33,0.32,0.33,...,0.35,0.36,0.37,0.24,0.27,0.28,0.29,0.30,0.31,2829
HOLT,0.16,0.19,0.23,0.28,0.29,0.31,0.29,0.28,0.24,0.30,...,0.31,0.32,0.33,0.22,0.24,0.25,0.26,0.26,0.27,2829
DAMPEN,0.16,0.18,0.22,0.27,0.29,0.30,0.27,0.29,0.25,0.28,...,0.34,0.33,0.35,0.21,0.24,0.25,0.25,0.26,0.28,2829
WINTER,0.16,0.19,0.24,0.28,0.30,0.31,0.28,0.28,0.25,0.30,...,0.31,0.33,0.32,0.22,0.25,0.26,0.26,0.26,0.27,2829
COMB S-H-D,0.16,0.19,0.23,0.27,0.28,0.30,0.27,0.28,0.25,0.30,...,0.32,0.32,0.33,0.21,0.24,0.25,0.25,0.26,0.27,2829
B-J auto,0.16,0.19,0.23,0.27,0.29,0.30,0.28,0.29,0.27,0.28,...,0.36,0.32,0.35,0.21,0.24,0.25,0.26,0.27,0.28,2829
AutoBox1,0.17,0.20,0.24,0.28,0.30,0.32,0.27,0.29,0.26,0.28,...,0.33,0.34,0.34,0.22,0.25,0.26,0.26,0.27,0.28,2829
AutoBox2,0.17,0.19,0.23,0.28,0.28,0.31,0.29,0.29,0.26,0.28,...,0.34,0.34,0.35,0.22,0.24,0.26,0.26,0.27,0.28,2829
AutoBox3,0.16,0.21,0.24,0.29,0.30,0.32,0.29,0.29,0.26,0.29,...,0.33,0.35,0.35,0.22,0.25,0.26,0.26,0.27,0.28,2829
